# DFS Example Code & Node Data Structure
Yoonsuck Choe
13 September 2021

In [1]:
DEBUG_FLAG=True
GOAL = 10

In [2]:
def debug(msg: str, obj: str = ""):
    """turn DEBUG_FLAG off to suppress messages

    Args:
        msg (str): message to display
        obj (str):
    """
    if (DEBUG_FLAG):
        print(msg, end="")
        print(obj)

In [3]:
def goalp(node):
    """goalp : check if it is a leaf (leaf value > GOAL, then goal found)

    Args:
        node (List[int]): node list

    Returns:
        Bool: False if node is not a leaf, node otherwise
    """
    if (not isinstance(node, int)):
        # not a leaf
        debug("  goalp: not a leaf node")
        return False

    else:
        # leaf node
        if (node > GOAL):
            debug("  goalp: MATCH")
            return node
        else:
            debug("  goalp: NO match")
            return False

# test:
# goalp([ [ 1 ,2 ] , 3] )
# goalp(11)

In [4]:
def expand(node):
    """given node, expand into children, and return list containing children

    Args:
        node (List[int]): node list

    Returns:
        List[int]: node expanded
    """
    if (isinstance(node,int)):
        return []
    else:
        '''
        If "node" = [ a, b, c ]
        then expanded items are a, b, c.
        Putting a, b, c in a list gives [ a, b, c ], which is the same as "node".
        '''
        return node

# test:
# expand([ 2 , [3 , 4 ], 5])
# expand(2)


In [5]:
def append_list(a: list, b: list) -> list:
    """Appends two lists together

    Args:
        a (list): originl list
        b (list): list to append

    Returns:
        the two input lists appended
    """
    ret = a
    ret.extend(b)
    return ret

In [6]:
def dfs(node: int) -> int:
    """Depth First Search, the iterative version

    Args:
        node (int): starting node

    Returns:
        int: goal node
    """
    # put node in empty nodelist
    node_list = [node]
    debug("Start: ", node_list)

    # MAIN LOOP: while node_list is not empty
    while (node_list):

        # A. visit: pop first node from list
        cur_node = node_list.pop(0)
        debug("Visiting: ", cur_node)

        # B. goal check
        if (goalp(cur_node)):

            # 1 goal found
            print("  MATCH = "+str(cur_node))
            return cur_node

        else:
            # 2 further explore

            # 2.1: expand node
            expanded = expand(cur_node)
            debug("  Expanded=", expanded)

            # 2.2: enqueue (change here to change search behavior)

            # for dfs:
            node_list = append_list(expanded, node_list)
            # for bfs: change to
            # node_list = append_list(node_list, expanded)

        # C. print current node_list
        debug("  New node list=", node_list)
        debug("")

In [7]:
# test: change the tree or change the goal (set GOAL at the top)
dfs([2 , [3 ,[7, 11 ]], [9, 20], 5])

Start: [[2, [3, [7, 11]], [9, 20], 5]]
Visiting: [2, [3, [7, 11]], [9, 20], 5]
  goalp: not a leaf node
  Expanded=[2, [3, [7, 11]], [9, 20], 5]
  New node list=[2, [3, [7, 11]], [9, 20], 5]

Visiting: 2
  goalp: NO match
  Expanded=[]
  New node list=[[3, [7, 11]], [9, 20], 5]

Visiting: [3, [7, 11]]
  goalp: not a leaf node
  Expanded=[3, [7, 11]]
  New node list=[3, [7, 11], [9, 20], 5]

Visiting: 3
  goalp: NO match
  Expanded=[]
  New node list=[[7, 11], [9, 20], 5]

Visiting: [7, 11]
  goalp: not a leaf node
  Expanded=[7, 11]
  New node list=[7, 11, [9, 20], 5]

Visiting: 7
  goalp: NO match
  Expanded=[]
  New node list=[11, [9, 20], 5]

Visiting: 11
  goalp: MATCH
  MATCH = 11


11

# Node Data Structure

Below are some ideas on how to make the simple example above more general
- use adjacency matrix to define graph, and conduct search on the graph
- each cell in the matrix is the distance
- column -> row connectivity

In [8]:
MAT = [[0, 10, 12,  9,  0,  0,  0,  0],  #A
       [0,  0,  0,  0, 25, 12,  0,  0],  #B
       [0,  0,  0,  0,  0, 18, 29,  0],  #C
       [0,  0,  5,  0,  0,  0, 25,  0],  #D
       [0,  0,  0,  0,  0,  7,  0, 20],  #E
       [0,  0,  0,  0,  0,  0,  9, 36],  #F
       [0,  0,  0,  0,  0,  0,  0, 20],  #G
       [0,  0,  0,  0,  0,  0,  0,  0]]  #H
'''
 The search graph looks like this:

 (0) --10--> (2) --20--> (3)
  |     _10__/            ^
  30  /                   |
  |  |                    |
  v  v                   /
  (1)_______5___________/

'''

# 2. heuristic values : states 0, 1, 2, 3
H = [41, 40, 37, 38, 17, 24, 19, 0]

In [9]:
class Node:
	"""Node class
	"""
	def __init__(self, state: int):
		"""Initialize a node

		Args:
			state (int): id for the node
		"""
		self.state: int = state
		self.h: int = H[state]
		self.cost: int = 0
		self.path: str = str(state)

	def __str__(self):
		return f"{{state={self.state}, cost={self.cost}, h={self.h}, path=({self.path})}}"

	def move(self, child_state: int):
		"""Make one move to child_state

		Args:
			child_state (int): Child to move to

		Returns:
			Node: resulting child node
		"""
		edge_cost = MAT[self.state][child_state]
		if (edge_cost > 0):
			child = Node(child_state)
			child.cost = self.cost + edge_cost
			child.path = f"{self.path} -> {child_state}"
			return child
		else:
			return []

	def expand(self) -> list:
		'''
		expand node and do all the book-keeping
		- returns node list
		'''

		node_list = []

		for child_state in range(0, len(H)):
			if (MAT[self.state][child_state] > 0):
				child_node = self.move(child_state)
				node_list.extend([child_node])

		return node_list

	def print_list(node_list: list) -> str:
		"""Pretty print the node list

		Args:
			node_list (list): node list to print

		Returns:
			str: list of nodes
		"""
		s = "["

		for n in node_list:
			s += str(n) + "; "
		s += "]\n"
		return s

	def sort(node_list: list, eval_fn: callable):
		"""Sorts a list of nodes based on the evaluation function.

		Args:
			node_list (list): A list of nodes to be sorted.
			eval_fn (callable): A function that takes a node as input and returns a value to determine the sorting order.

		Returns:
			list[Node]: A sorted list of nodes.
		"""
		return sorted(node_list, key=eval_fn)

In [10]:
def gbfs(start, goal) -> Node:
    pq = [Node(start)]
    debug(f"Start: {Node.print_list(pq)}")

    while (pq):
        cur_node = pq.pop(0)
        debug(f"Visiting: {str(cur_node)}")

        if (cur_node.state == goal):
            print(f"  MATCH = {str(cur_node)}")
            return cur_node

        else:
            expanded = cur_node.expand()
            debug(f"  Expanded={Node.print_list(expanded)}")

            pq.extend(expanded)

        pq = Node.sort(pq, lambda node: node.h)
        debug(f"  New node list={Node.print_list(pq)}")
        debug("", "")

In [11]:
print(gbfs(0, 7))

Start: [{state=0, cost=0, h=41, path=(0)}; ]

Visiting: {state=0, cost=0, h=41, path=(0)}
  Expanded=[{state=1, cost=10, h=40, path=(0 -> 1)}; {state=2, cost=12, h=37, path=(0 -> 2)}; {state=3, cost=9, h=38, path=(0 -> 3)}; ]

  New node list=[{state=2, cost=12, h=37, path=(0 -> 2)}; {state=3, cost=9, h=38, path=(0 -> 3)}; {state=1, cost=10, h=40, path=(0 -> 1)}; ]


Visiting: {state=2, cost=12, h=37, path=(0 -> 2)}
  Expanded=[{state=5, cost=30, h=24, path=(0 -> 2 -> 5)}; {state=6, cost=41, h=19, path=(0 -> 2 -> 6)}; ]

  New node list=[{state=6, cost=41, h=19, path=(0 -> 2 -> 6)}; {state=5, cost=30, h=24, path=(0 -> 2 -> 5)}; {state=3, cost=9, h=38, path=(0 -> 3)}; {state=1, cost=10, h=40, path=(0 -> 1)}; ]


Visiting: {state=6, cost=41, h=19, path=(0 -> 2 -> 6)}
  Expanded=[{state=7, cost=61, h=0, path=(0 -> 2 -> 6 -> 7)}; ]

  New node list=[{state=7, cost=61, h=0, path=(0 -> 2 -> 6 -> 7)}; {state=5, cost=30, h=24, path=(0 -> 2 -> 5)}; {state=3, cost=9, h=38, path=(0 -> 3)}; {state

In [12]:
def a_star(start, goal) -> Node:
    pq = [Node(start)]
    debug(f"Start: {Node.print_list(pq)}")

    while (pq):
        cur_node = pq.pop(0)
        debug(f"Visiting: {str(cur_node)}")

        if (cur_node.state == goal):
            print(f"  MATCH = {str(cur_node)}")
            return cur_node

        else:
            expanded = cur_node.expand()
            debug(f"  Expanded={Node.print_list(expanded)}")

            pq.extend(expanded)

        pq = Node.sort(pq, lambda node: node.h + node.cost)
        debug(f"  New node list={Node.print_list(pq)}")
        debug("", "")

In [13]:
print(a_star(0, 7))

Start: [{state=0, cost=0, h=41, path=(0)}; ]

Visiting: {state=0, cost=0, h=41, path=(0)}
  Expanded=[{state=1, cost=10, h=40, path=(0 -> 1)}; {state=2, cost=12, h=37, path=(0 -> 2)}; {state=3, cost=9, h=38, path=(0 -> 3)}; ]

  New node list=[{state=3, cost=9, h=38, path=(0 -> 3)}; {state=2, cost=12, h=37, path=(0 -> 2)}; {state=1, cost=10, h=40, path=(0 -> 1)}; ]


Visiting: {state=3, cost=9, h=38, path=(0 -> 3)}
  Expanded=[{state=2, cost=14, h=37, path=(0 -> 3 -> 2)}; {state=6, cost=34, h=19, path=(0 -> 3 -> 6)}; ]

  New node list=[{state=2, cost=12, h=37, path=(0 -> 2)}; {state=1, cost=10, h=40, path=(0 -> 1)}; {state=2, cost=14, h=37, path=(0 -> 3 -> 2)}; {state=6, cost=34, h=19, path=(0 -> 3 -> 6)}; ]


Visiting: {state=2, cost=12, h=37, path=(0 -> 2)}
  Expanded=[{state=5, cost=30, h=24, path=(0 -> 2 -> 5)}; {state=6, cost=41, h=19, path=(0 -> 2 -> 6)}; ]

  New node list=[{state=1, cost=10, h=40, path=(0 -> 1)}; {state=2, cost=14, h=37, path=(0 -> 3 -> 2)}; {state=6, cost=34,